In [17]:
import pandas as pd


def set_delay_type(row):  
    if row['ARRIVAL_DELAY'] <= 15:
        return 'No Delay'
    elif row['ARRIVAL_DELAY'] > 15 and row['ARRIVAL_DELAY'] <= 60:
        return 'Delayed'
    else:
        return 'Heavy Delay'


def set_distance_type(row):  
    if row['DISTANCE'] <= 1500:
        return 'Short'
    elif row['DISTANCE'] > 1500 and row['DISTANCE'] <= 3500:
        return 'Median'
    else:
        return 'Long'


def remove_irrelevant_columns(df):
    columns_to_remove = ['DIVERTED',
                        'CANCELLED',
                        'CANCELLATION_REASON',
                        'AIR_SYSTEM_DELAY',
                        'SECURITY_DELAY',
                        'AIRLINE_DELAY',
                        'LATE_AIRCRAFT_DELAY',
                        'WEATHER_DELAY',
                        'TAIL_NUMBER']
    for col in columns_to_remove:
        df.pop(col)
    return df


def remove_unnamed(df):
    for col in df.columns:
        if 'Unnamed:' in col:
            df.pop(col)
    return df


def remove_irrelevant_airports(df, treshold):
    """
    Removes airport rowns from the dict by their numberic value and apearance
    This function purpose is to reduce fie size for BigMl training limits
    """
    df.replace({'ATL':'TLV'}, inplace=True)
    df = df[(df['ORIGIN_AIRPORT'].str.contains('TLV') == True) | (df['DESTINATION_AIRPORT'].str.contains('TLV') == True)]
    values_c = df['ORIGIN_AIRPORT'].value_counts()
    values_c_dict = dict(values_c)
    for k, v in values_c_dict.items():
        if v < treshold:
            df = df[(df['ORIGIN_AIRPORT'].str.contains(k) == False) & (df['DESTINATION_AIRPORT'].str.contains(k) == False)]
    return df


def save_csv(df, path):
    df.to_csv(path, index=False)


def replace_airports():
    airport_replace_dict = {'LGA':'MLA', 'MCO': 'ADD', 'FLL': 'FCO'}
    df.replace(airport_replace_dict, inplace=True)


def preprocess_df(df, path):
    df.replace({'ATL':'TLV'}, inplace=True)
    df = remove_irrelevant_columns(df)
    df = remove_unnamed(df)
    df = remove_irrelevant_airports(df, 5100)
    df['DELAY_TYPE'] = df.apply(lambda row: set_delay_type(row), axis=1)
    df['DISTANCE_TYPE'] = df.apply(lambda row: set_distance_type(row), axis=1)
    save_csv(df, path)
    return df


pd.set_option('display.max_columns', None)
filepath = './../../flights.csv/flights.csv'
result_path = './../Datasets/flights_for_BigML.csv'

df = pd.read_csv(filepath)
df = remove_irrelevant_airports(df, 5100)
save_csv(df, filepath)
# df

df = preprocess_df(df, result_path)
# df['ORIGIN_AIRPORT'].value_counts()

# we get ATL   - 346836

# Filter out any non TLV flight
df




/tmp/ipykernel_19524/3233654938.py:83: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DELAY_TYPE,DISTANCE_TYPE
124,2015,1,1,4,DL,1890,DFW,TLV,545,603.0,18.0,13.0,616.0,124.0,104.0,86.0,731,842.0,5.0,849,847.0,-2.0,No Delay,Short
258,2015,1,1,4,DL,904,LGA,TLV,600,601.0,1.0,11.0,612.0,159.0,126.0,111.0,762,803.0,4.0,839,807.0,-32.0,No Delay,Short
259,2015,1,1,4,DL,977,ORD,TLV,600,602.0,2.0,19.0,621.0,127.0,104.0,80.0,606,841.0,5.0,907,846.0,-21.0,No Delay,Short
268,2015,1,1,4,DL,2154,LAX,TLV,600,559.0,-1.0,12.0,611.0,250.0,231.0,213.0,1947,1244.0,6.0,1310,1250.0,-20.0,No Delay,Median
444,2015,1,1,4,DL,327,PHL,TLV,610,608.0,-2.0,16.0,624.0,155.0,126.0,103.0,666,807.0,7.0,845,814.0,-31.0,No Delay,Short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5818874,2015,12,31,4,DL,1711,TLV,DFW,2225,2220.0,-5.0,16.0,2236.0,147.0,139.0,118.0,731,2334.0,5.0,2352,2339.0,-13.0,No Delay,Short
5818928,2015,12,31,4,WN,3584,TLV,PHL,2245,2243.0,-2.0,10.0,2253.0,120.0,97.0,82.0,666,15.0,5.0,45,20.0,-25.0,No Delay,Short
5818962,2015,12,31,4,AA,1116,LAX,TLV,2300,2256.0,-4.0,15.0,2311.0,253.0,237.0,207.0,1947,538.0,15.0,613,553.0,-20.0,No Delay,Median
5818996,2015,12,31,4,F9,1478,LAX,TLV,2320,2309.0,-11.0,31.0,2340.0,252.0,295.0,211.0,1947,611.0,53.0,632,704.0,32.0,Delayed,Median


In [14]:
# df['ORIGIN_AIRPORT'].value_counts()
# or df['DESTINATION_AIRPORT'].str.contains('TLV')==True
# df2 = df[df['AIRLINE'].str.contains('ELY') == True]
# df.to_csv('./../flights.csv/flights.csv')


# df.to_csv('./../flights.csv/flights.csv')
# df = preprocess_df(df, result_path)
df['ORIGIN_AIRPORT'].value_counts()
# df = remove_irrelevant_airports(df, 500)
# save_csv(df, filepath)
df = df.iloc[: , 1:]
df
df.to_csv("./tmp.csv")


In [ ]:
# df2 = 
df[(df['ORIGIN_AIRPORT'].str.contains('TPA') == True) | (df['DESTINATION_AIRPORT'].str.contains('TPA') == True)]
# df2['DISTANCE']
# airport_replace_dict = {'LGA':'MLA', 'MCO': 'ADD', 'FLL': 'FCO'}
df


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,2355.0,-4.0,22.0,17.0,320.0,298.0,272.0,2611,749.0,4.0,819,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,2355.0,-4.0,17.0,12.0,227.0,215.0,195.0,1617,427.0,3.0,446,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,2350.0,-9.0,17.0,7.0,221.0,222.0,197.0,1598,424.0,8.0,440,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,2353.0,-6.0,10.0,3.0,161.0,157.0,144.0,1189,327.0,3.0,340,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
save_csv(df, result_path)